# SPRI Bead Protocol specifically for OT-2

## Assumptions:
- 8 channel pipette head
- PCR of 20 µL volume in a 96 wellplate
- PCR wells are filled from left to right, top to bottom without gaps?

In [ ]:
# (optional) dont save output when adding to git.
# ! pip install nbstripout
# ! nbstripout --install


# save all logs to output file
# TODO: logs to output file.

In [ ]:
# Imports

from pylabrobot.resources.opentrons import opentrons_96_tiprack_300ul, OTDeck, opentrons_96_tiprack_300ul
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend, OpentronsBackend
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.resources import Plate, TipRack


import time

In [ ]:
# Settings          -> soon to go into a .json options file

always_use_diff_plate = False       # if under half of a plate, use the second half of the 96w plate
magnet_module = True                # if automatic magnet module
elute_vol = 20                      # 20µL

In [ ]:
# Global positioning variables for objects
tip_rack_0 = opentrons_96_tiprack_300ul(name='tip_rack_0')
tip_rack_1 = opentrons_96_tiprack_300ul(name='tip_rack_1')

tip_racks = [tip_rack_0, tip_rack_1]

mag_module = None       # TODO: define magnetic module
pcr_plate = None
pcr_plate_2 = None          # used if must switch out for a dna_ceil
purified_plate = None
reagent_plate = None
# cols of reagent plate
bead_col = 1
ethanol_col = 2
elute_col = 3
waste_col = 4

lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=OTDeck())



# Deck config
lh.deck.assign_child_at_slot(tip_rack_0, 11)
lh.deck.assign_child_at_slot(tip_rack_1, 10)

lh.deck.assign_child_at_slot(reagent_plate, 1)
lh.deck.assign_child_at_slot(purified_plate, 3)
lh.deck.assign_child_at_slot(mag_module, 4)
mag_module.add_plate(pcr_plate)

# TODO: setup deck



In [ ]:
cptl_alphabet = [chr(i) for i in range(65, 91)]
print(cptl_alphabet)
print(cptl_alphabet[:3])

In [ ]:
def reload_tips():
    print('Please reload all tip racks, the tips have been moved into apropriate positions for easy filling!')
    input('Press enter once all tip_racks have been refilled.')
    for tip_rack in tip_racks:
        tip_rack.fill()

In [ ]:
class TipRackTracker():
    def __init__(self):
        self.tip_racks = tip_racks
        self.curr_tip_col = 0
        self.rack_index = 0

    def get_tip_spots(num_tips: int = 8):
        curr_tip_col += 1
        if curr_tip_col > 12:
            curr_tip_col = 1
            rack_index += 1
            if rack_index >= len(tip_racks):
                 # TODO: fix so that unused tips do not go to waste => lh.consolodate_tip_inventory(tip_racks=tip_racks)                    
                reload_tips()
        spots = f'{cptl_alphabet[:num_tips]}{curr_tip_col}'
        return tip_racks[rack_index][spots]

In [ ]:
async def spri_protocol_with_magnet_module(sample_wells: list[str], dna_floor: int, dna_ceil: int) -> dict[str: str]:
    '''
    Runs an SPRI bead separation protocol for DNA within the selected range dna_floor <= x <= dna_ceil.

    This protocol is designed for PCR that has been run on a 96w PCR plate.

    Args:
        sample_wells (list[str]): sample wells to purify. Ex: ['A1', 'B3', ..., 'H1']
        dna_floor (int): the lower end of dna size to select for. None if unwanted.
        dna_ceil (int): the highest end of dna size to select for. None if unwanted.

    Returns:
        dict{str:str}: {well on purified plate: well on pcr_plate}
    '''
    async def mix(plate: Plate, positions: list[str], vols: int, cycles: int = 10):
        ''' 
        helper method to mix a set of up to 8 wells

        Args:
            plate (Plate):
            positions (list[str]):
            vols (int):
            cycles (int):
        '''
        if len(positions) > 8:
            raise ValueError(f'Too many positions to mix, given {len(positions)} values where max is 8.')

        for _ in cycles:
            lh.aspirate(plate[positions], vols=vols)
            lh.dispense(plate[positions], vols=vols)

    trt = TipRackTracker()

    # bead volume logic:
    bead_ceil_vol = 20            # 1x until math is established
    bead_floor_vol = 20           # 1x until math is established

    # initialization
    if not always_use_diff_plate:
        is_under_half = True if ((len(sample_wells) / 48) <= 1) else False        # Can use the second half of the plate, start at second half and if wells filled, push them to the next available, when overflowing continue to the first side of the plate. 


    ethanol_wash_vol = 180      # µL

    well_cols = []

    for i in range(1, 13):
        col = []
        # for all cols on a plate
        for well in sample_wells:
            if well[1:] == str(i):
                col.append(well)

        well_cols.append(col)

    plate = pcr_plate

    if dna_ceil is not None:
        # have to do a selection for both sides, with saving of supernatant after first bead mixing step.
        for col in well_cols:
            # loop over active wells
            num_wells = len(col)           # how many pipet heads to use (should be 8 until partial rows)

            # add beads to PCR mixture
            await lh.pick_up_tips(tip_spots=trt.get_tip_spots(num_tips=num_wells))
            await lh.aspirate(reagent_plate[bead_col], vols=bead_vol_ceil)
            await lh.dispense(plate[col], vols=bead_vol_ceil)
            await lh.return_tips()



        plate = pcr_plate_2         # TODO: less than half logic



    # TODO: after dna_ceil has been dealt with and liquid transferred to new wells:
    for col in well_cols:
        num_wells = len(col)
        # loop over active wells
        # TODO: set tip_positions
        # instructions
        # add SPRI & solution to wells w/DNA -->> assume that DNA is already in PCR plate in 20µL aliqots. 
        await lh.pick_up_tips(tip_spots=trt.get_tip_spots(num_tips=num_wells))
        await lh.aspirate(reagent_plate[bead_col], vols=bead_vol_floor)
        await lh.dispense(plate[col], vols=bead_vol_floor)
        # mix 10x
        await mix(locations=col, vols=bead_vol_floor)
        await lh.return_tips()
    # incubate 1 min -> allow last row to sit for one minute before the magnet initializes 
    time.sleep(60)
    # magnet on
    await mag_module.engage()
    # incubate 1.5 min for first col
    time.sleep(90)
    for col in well_cols:
        # aspirate supernatant
        await lh.pick_up_tips(tip_spots=trt.get_tip_spots(num_tips=num_wells))
        await lh.aspirate(pcr_plate[col], vols=ethanol_wash_vol)
        await lh.return_tips()
        # ethanol wash 3x   ( with mix steps and incubation steps)
        for _ in range(3):
            await lh.pick_up_tips(tip_spots=trt.get_tip_spots(num_tips=num_wells))
            await lh.aspirate(reagent_plate[ethanol_col], vols=ethanol_wash_vol)
            await lh.dispense(plate[col], vols=ethanol_wash_vol)
            await mix(plate=plate, positions=col, vols=ethanol_wash_vol/2, cycles=2)
            await lh.aspirate(plate[col], vols=ethanol_wash_vol)
            await lh.dispense(reagent_plate[waste_col], vols=ethanol_wash_vol)
            await lh.return_tips()
        # elution buffer addition (on top of beads?)
        await lh.pick_up_tips(tip_spots=trt.get_tip_spots(num_tips=num_wells))
        await lh.aspirate(resources=reagent_plate[elute_col]*8, vols=elute_vol)
        await lh.dispense(plate[col], vols=elute_vol)
        await lh.return_tips()
    # incubate 1 min
    # magnet on
    time.sleep(60)
    m
        # aspirate and save supernatant
    
    return None

In [ ]:
sample_wells = None
dna_floor = 0
dna_ceil = None

if magnet_module:
    spri_protocol_with_magnet_module(sample_wells=None, dna_floor=dna_floor, dna_ceil=dna_ceil)